In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json

Geojson data sources:
- https://eric.clst.org/tech/usgeojson/
- https://github.com/OpenDataDE/State-zip-code-GeoJSON

In [2]:
# uhh these files get pretty big pretty quick
with open('data/geojson/state/gz_2010_us_040_00_20m.json', 'r') as f:
    states_geojson = json.load(f)

with open('data/geojson/zipcode/ca_california_zip_codes_geo.min.json', 'r') as f:
    ca_zipcode_geojson = json.load(f)

In [3]:
state_abbreviation = pd.read_csv('data/state_abbreviation.csv')
state_abbreviation_dict = state_abbreviation.set_index('Abbreviation')['Name'].to_dict()

In [4]:
rental_df_raw = pd.read_parquet('data/clean_housing_data.parquet')

In [5]:
# To avoid importing repeatedly
rental_df = rental_df_raw

In [6]:
rental_df.loc[:, 'state'] = rental_df.loc[:, 'state'].replace(state_abbreviation_dict)

In [7]:
# Trying to limit data set to only a time step for now
initialdate = rental_df['date'].iloc[0]
initialrental_df = rental_df[rental_df['date'] == initialdate]

In [8]:
initialrentbystate = initialrental_df.loc[:, ['state', 'market_rent']].groupby('state').mean()

In [9]:
initialrentbystate

,market_rent
state,
Arizona,2288.132353
California,3135.859504
Colorado,2535.089109
Florida,2728.968701
Georgia,2186.465608
Illinois,2473.694444
Minnesota,2376.758621
Nevada,2345.891667
North Carolina,2083.202381


In [10]:
# This figure might take a while and honestly doesn't really show anything interesting...
# fig = go.Figure(
#     go.Choroplethmapbox(
#         geojson= states_geojson,
#         featureidkey= "properties.NAME",
#         locations= initialrentbystate.index,
#         z= initialrentbystate['market_rent'],
#         marker_opacity=0.5, 
#         marker_line_width=0,
#     )
# )

# fig.update_layout(mapbox_style="carto-positron",
#                   mapbox_zoom=3, mapbox_center = {"lat": 37.0902, "lon": -95.7129})
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# fig.show()

In [11]:
# Trying to get a breakdown of a specific state
initialrental_df_ca = initialrental_df[initialrental_df['state'] == 'California']
initialrent_ca = initialrental_df_ca.loc[:, ['zipcode', 'market_rent']].groupby('zipcode').mean()

In [12]:
# initialrent_ca
initialrent_ca.sort_values('market_rent')

,market_rent
zipcode,
95993.0,2295.0
95833.0,2295.0
92562.0,2295.0
95824.0,2299.0
95240.0,2395.0
...,...
91390.0,4115.0
92506.0,4219.0
91320.0,4295.0


In [13]:
# This figure takes a really long time, there are a lot of zip codes, could just search the zip codes manually ...
# Uhhhhh this figure breaks stuff because of how laggy it is i think...
# I guess its useful as a visualization of where things are but it probably should just be written to html and viewed separately
# fig = go.Figure(
#     go.Choroplethmapbox(
#         geojson= ca_zipcode_geojson,
#         featureidkey= "properties.ZCTA5CE10",
#         locations= initialrent_ca.index,
#         z= initialrent_ca['market_rent'],
#         marker_opacity=0.5, 
#         marker_line_width=0,
#     )
# )

# fig.update_layout(mapbox_style="carto-positron",
#                   mapbox_zoom=3, mapbox_center = {"lat": 37.0902, "lon": -95.7129})
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# fig.write_html("reports/ca_zip_market_rent.html")